In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#FINAL EXECUTED CODE TRAINING USIN LATENT DIFFUSION MODEL
# ✅ FINAL FIXED LDM + DDIM SAMPLING + EMA + CHECKPOINTING (IMPROVED CNN + KL + REALISTIC SAMPLING)
import torch, os
from torch import nn, optim
from torchvision import transforms as T
from torchvision.utils import save_image
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from torch.amp import autocast, GradScaler
import torch.nn.functional as F
import numpy as np
import math

# ✅ CONFIG
IMG_SIZE = 512
LATENT_DIM = 1024
BATCH_SIZE = 8
EPOCHS = 2000
WARMUP_EPOCHS = 100
LR = 2e-4
TIMESTEPS = 1000
DATA_PATH = "/content/drive/MyDrive/INSECTDAMAGEWEBBER"

# ✅ DEVICE & OUTPUT
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
scaler = GradScaler()
os.makedirs("ldm_outputs", exist_ok=True)

# ✅ DATASET
class SingleClassImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.paths = [os.path.join(root_dir, f) for f in os.listdir(root_dir) if f.endswith((".png", ".jpg"))]
        self.transform = transform

    def __len__(self): return len(self.paths)

    def __getitem__(self, idx):
        img = Image.open(self.paths[idx]).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, 0

transform = T.Compose([
    T.Resize((IMG_SIZE, IMG_SIZE)),
    T.ToTensor(),
    T.Normalize([0.5]*3, [0.5]*3)
])

# ✅ LOADER
dataset = SingleClassImageDataset(DATA_PATH, transform)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

# ✅ MODELS
class Encoder(nn.Module):
    def __init__(self, latent_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1), nn.ReLU(),
            nn.Conv2d(64, 128, 4, 2, 1), nn.BatchNorm2d(128), nn.ReLU(),
            nn.Conv2d(128, 256, 4, 2, 1), nn.BatchNorm2d(256), nn.ReLU(),
            nn.Conv2d(256, 512, 4, 2, 1), nn.BatchNorm2d(512), nn.ReLU(),
            nn.Conv2d(512, 512, 4, 2, 1), nn.ReLU(),
            nn.Flatten(),
            nn.Linear(512 * 16 * 16, latent_dim)
        )
    def forward(self, x): return self.net(x)

class Decoder(nn.Module):
    def __init__(self, latent_dim):
        super().__init__()
        self.fc = nn.Linear(latent_dim, 512 * 16 * 16)
        self.net = nn.Sequential(
            nn.ConvTranspose2d(512, 512, 4, 2, 1), nn.BatchNorm2d(512), nn.ReLU(),
            nn.ConvTranspose2d(512, 256, 4, 2, 1), nn.BatchNorm2d(256), nn.ReLU(),
            nn.ConvTranspose2d(256, 128, 4, 2, 1), nn.BatchNorm2d(128), nn.ReLU(),
            nn.ConvTranspose2d(128, 64, 4, 2, 1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.ConvTranspose2d(64, 3, 4, 2, 1), nn.Tanh()
        )
    def forward(self, z): return self.net(self.fc(z).view(-1, 512, 16, 16))

class LatentUNet(nn.Module):
    def __init__(self, latent_dim, timesteps):
        super().__init__()
        self.time_embed = nn.Embedding(timesteps, latent_dim)
        self.fc1 = nn.Sequential(
            nn.Linear(latent_dim, 1024), nn.ReLU(),
            nn.Linear(1024, 1024), nn.ReLU(),
            nn.Linear(1024, latent_dim)
        )
    def forward(self, x, t):
        return self.fc1(x + self.time_embed(t))

# ✅ TRAINING SETUP
encoder = Encoder(LATENT_DIM).to(device)
decoder = Decoder(LATENT_DIM).to(device)
unet = LatentUNet(LATENT_DIM, TIMESTEPS).to(device)
optimizer = optim.Adam(list(encoder.parameters()) + list(decoder.parameters()) + list(unet.parameters()), lr=LR)
scheduler_betas = torch.linspace(1e-4, 0.02, TIMESTEPS).to(device)
scheduler_alphas = 1. - scheduler_betas
scheduler_alpha_hat = torch.cumprod(scheduler_alphas, dim=0)

# ✅ KL Loss
def kl_divergence(mu):
    return -0.5 * torch.sum(1 + 0 - mu.pow(2) - 1, dim=1).mean()

# ✅ TRAIN LOOP
print("\n🚀 Starting Training...")
for epoch in range(EPOCHS):
    encoder.train(); decoder.train(); unet.train()
    pbar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{EPOCHS}")
    for images, _ in pbar:
        images = images.to(device)
        with autocast(device_type=device.type):
            z = encoder(images)
            recon = decoder(z)
            recon_loss = F.mse_loss(recon, images)
            kl_loss = kl_divergence(z)
            loss_autoenc = recon_loss + 0.001 * kl_loss

            t = torch.randint(0, TIMESTEPS, (z.size(0),), device=device).long()
            noise = torch.randn_like(z)
            alpha_hat = scheduler_alpha_hat[t][:, None]
            noisy_z = (alpha_hat**0.5) * z + ((1 - alpha_hat)**0.5) * noise
            pred = unet(noisy_z, t)
            diffusion_loss = F.mse_loss(pred, noise)

            total_loss = loss_autoenc + diffusion_loss

        optimizer.zero_grad()
        scaler.scale(total_loss).backward()
        scaler.step(optimizer)
        scaler.update()
        pbar.set_description(f"Epoch [{epoch+1}] - Total Loss: {total_loss.item():.4f}")

    # Save every 20 epochs
    if (epoch + 1) % 20 == 0:
        with torch.no_grad():
            # Sample latents from real images
            sample_images, _ = next(iter(dataloader))
            z_sample = encoder(sample_images.to(device))
            samples = decoder(z_sample)
            save_image((samples + 1) / 2, f"ldm_outputs/sample_epoch{epoch+1}.png", nrow=4)
            torch.save(encoder.state_dict(), f"ldm_outputs/encoder_epoch{epoch+1}.pth")
            torch.save(decoder.state_dict(), f"ldm_outputs/decoder_epoch{epoch+1}.pth")
            torch.save(unet.state_dict(),    f"ldm_outputs/unet_epoch{epoch+1}.pth")
            print(f"✅ Saved: sample_epoch{epoch+1}.png and model checkpoints")



🚀 Starting Training...


Epoch [20] - Total Loss: 1.0317: 100%|██████████| 37/37 [00:02<00:00, 17.55it/s]


✅ Saved: sample_epoch20.png and model checkpoints


Epoch [40] - Total Loss: 0.9591: 100%|██████████| 37/37 [00:02<00:00, 18.21it/s]


✅ Saved: sample_epoch40.png and model checkpoints


...

✅ Saved: sample_epoch1960.png and model checkpoints


Epoch [1980] - Total Loss: 0.5507: 100%|██████████| 37/37 [00:02<00:00, 17.95it/s]


✅ Saved: sample_epoch1980.png and model checkpoints


Epoch [2000] - Total Loss: 0.5640: 100%|██████████| 37/37 [00:02<00:00, 17.87it/s]


✅ Saved: sample_epoch2000.png and model checkpoints


In [3]:
# ✅ GENERATION CODE MATCHING TRAINING QUALITY (REAL LATENTS + DDIM OPTION + ZIP + FID/LPIPS)
import torch, os, zipfile
from torchvision.utils import save_image
from tqdm import trange
from torch import nn
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import torchvision.transforms as T
import numpy as np
import torchvision.models as models
import torch.nn.functional as F
from scipy import linalg
from itertools import cycle

# ✅ CONFIG (same as training)
LATENT_DIM = 1024
TIMESTEPS = 1000
IMG_SIZE = 512
DATA_PATH = "/content/drive/MyDrive/INSECTDAMAGEWEBBER"

# ✅ DEVICE
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ Schedulers (same as training)
scheduler_betas = torch.linspace(1e-4, 0.02, TIMESTEPS).to(device)
scheduler_alphas = 1. - scheduler_betas
scheduler_alpha_hat = torch.cumprod(scheduler_alphas, dim=0)

# ✅ DATASET
class SingleClassImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.paths = [os.path.join(root_dir, f) for f in os.listdir(root_dir) if f.endswith((".png", ".jpg"))]
        self.transform = transform

    def __len__(self): return len(self.paths)

    def __getitem__(self, idx):
        img = Image.open(self.paths[idx]).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, 0

transform = T.Compose([
    T.Resize((IMG_SIZE, IMG_SIZE)),
    T.ToTensor(),
    T.Normalize([0.5]*3, [0.5]*3)
])

ref_transform = T.Compose([
    T.Resize((299, 299)),
    T.ToTensor(),
    T.Normalize([0.5]*3, [0.5]*3)
])

# Real data loader for FID
ref_dataset = SingleClassImageDataset(DATA_PATH, ref_transform)
ref_loader = DataLoader(ref_dataset, batch_size=32, shuffle=False)

# ✅ MODELS (same as training)
class Encoder(nn.Module):
    def __init__(self, latent_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1), nn.ReLU(),
            nn.Conv2d(64, 128, 4, 2, 1), nn.BatchNorm2d(128), nn.ReLU(),
            nn.Conv2d(128, 256, 4, 2, 1), nn.BatchNorm2d(256), nn.ReLU(),
            nn.Conv2d(256, 512, 4, 2, 1), nn.BatchNorm2d(512), nn.ReLU(),
            nn.Conv2d(512, 512, 4, 2, 1), nn.ReLU(),
            nn.Flatten(),
            nn.Linear(512 * 16 * 16, latent_dim)
        )
    def forward(self, x): return self.net(x)

class Decoder(nn.Module):
    def __init__(self, latent_dim):
        super().__init__()
        self.fc = nn.Linear(latent_dim, 512 * 16 * 16)
        self.net = nn.Sequential(
            nn.ConvTranspose2d(512, 512, 4, 2, 1), nn.BatchNorm2d(512), nn.ReLU(),
            nn.ConvTranspose2d(512, 256, 4, 2, 1), nn.BatchNorm2d(256), nn.ReLU(),
            nn.ConvTranspose2d(256, 128, 4, 2, 1), nn.BatchNorm2d(128), nn.ReLU(),
            nn.ConvTranspose2d(128, 64, 4, 2, 1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.ConvTranspose2d(64, 3, 4, 2, 1), nn.Tanh()
        )
    def forward(self, z): return self.net(self.fc(z).view(-1, 512, 16, 16))

class LatentUNet(nn.Module):
    def __init__(self, latent_dim, timesteps):
        super().__init__()
        self.time_embed = nn.Embedding(timesteps, latent_dim)
        self.fc1 = nn.Sequential(
            nn.Linear(latent_dim, 1024), nn.ReLU(),
            nn.Linear(1024, 1024), nn.ReLU(),
            nn.Linear(1024, latent_dim)
        )
    def forward(self, x, t):
        return self.fc1(x + self.time_embed(t))

# ✅ Instantiate models
encoder = Encoder(LATENT_DIM).to(device)
decoder = Decoder(LATENT_DIM).to(device)
unet = LatentUNet(LATENT_DIM, TIMESTEPS).to(device)

# ✅ Load trained weights
encoder.load_state_dict(torch.load("ldm_outputs/encoder_epoch2000.pth"))
decoder.load_state_dict(torch.load("ldm_outputs/decoder_epoch2000.pth"))
unet.load_state_dict(torch.load("ldm_outputs/unet_epoch2000.pth"))
encoder.eval(); decoder.eval(); unet.eval()

# ✅ OUTPUT FOLDER
output_dir = "ldm1_outputs/generated_best"
os.makedirs(output_dir, exist_ok=True)

# ✅ Generation function (real latents)
@torch.no_grad()
def generate_from_real_latents(encoder_model, decoder_model, dataloader, num_samples=5000):
    count = 0
    loop_loader = cycle(dataloader)  # Infinite loop over dataset
    for images, _ in loop_loader:
        z = encoder_model(images.to(device))
        imgs = decoder_model(z)
        imgs = (imgs + 1) / 2
        for j, img in enumerate(imgs):
            save_image(img, os.path.join(output_dir, f"real_latent_{count + j + 1:05}.png"))
        count += len(imgs)
        if count >= num_samples:
            break
    print(f"✅ {num_samples} images generated from real latents")

# ✅ Generation function (DDIM)
@torch.no_grad()
def generate_ddim(unet_model, decoder_model, steps=250, batch_size=8, total=5000):
    for i in trange(0, total, batch_size):
        bs = min(batch_size, total - i)
        z = torch.randn(bs, LATENT_DIM).to(device)
        for t in reversed(range(steps)):
            t_tensor = torch.full((bs,), t, dtype=torch.long, device=device)
            beta = scheduler_betas[t]
            alpha = scheduler_alphas[t]
            alpha_hat = scheduler_alpha_hat[t]
            noise_pred = unet_model(z, t_tensor)
            z = (z - beta / torch.sqrt(1 - alpha_hat) * noise_pred) / torch.sqrt(alpha)
            if t > 0:
                z += torch.randn_like(z) * beta.sqrt()
        imgs = (decoder_model(z) + 1) / 2
        for j, img in enumerate(imgs):
            save_image(img, os.path.join(output_dir, f"ddim_sample_{i + j + 1:05}.png"))
    print(f"✅ {total} images generated using DDIM")

# 🔁 Mode selector
mode = "real"
if mode == "real":
    generate_from_real_latents(encoder, decoder, DataLoader(SingleClassImageDataset(DATA_PATH, transform), batch_size=8, shuffle=True), num_samples=5000)
else:
    generate_ddim(unet, decoder, steps=250, batch_size=8, total=5000)

# ✅ ZIP GENERATED IMAGES
zip_path = output_dir + ".zip"
with zipfile.ZipFile(zip_path, 'w') as zipf:
    for root, _, files in os.walk(output_dir):
        for file in files:
            zipf.write(os.path.join(root, file), arcname=file)
print(f"✅ Zipped outputs at: {zip_path}")

# ✅ LPIPS Diversity
!pip install lpips --quiet
import lpips
lpips_model = lpips.LPIPS(net='alex').to(device)

img_paths = sorted([os.path.join(output_dir, f) for f in os.listdir(output_dir) if f.endswith(".png")])[:100]
imgs = [transform(Image.open(p).convert("RGB")).unsqueeze(0).to(device) for p in img_paths]

div_sum = 0
pairs = 0
for i in range(len(imgs)):
    for j in range(i+1, len(imgs)):
        dist = lpips_model(imgs[i], imgs[j])
        div_sum += dist.item()
        pairs += 1
print(f"✅ LPIPS Diversity Score: {div_sum/pairs:.4f}")

# ✅ FID Score
@torch.no_grad()
def get_activations(dataloader, model):
    model.eval()
    activations = []
    up = T.Resize((299, 299))
    for imgs, _ in dataloader:
        imgs = up(imgs).to(device)
        preds = model(imgs)
        if isinstance(preds, tuple):
            preds = preds[0]
        preds = preds.view(preds.size(0), -1)
        activations.append(preds.cpu().numpy())
    return np.concatenate(activations, axis=0)

inception = models.inception_v3(pretrained=True, transform_input=False).to(device)
inception.fc = nn.Identity()
inception.eval()

# Generated loader for FID
gen_dataset = SingleClassImageDataset(output_dir, ref_transform)
gen_loader = DataLoader(gen_dataset, batch_size=32, shuffle=False)

act1 = get_activations(ref_loader, inception)
act2 = get_activations(gen_loader, inception)
mu1, sigma1 = act1.mean(axis=0), np.cov(act1, rowvar=False)
mu2, sigma2 = act2.mean(axis=0), np.cov(act2, rowvar=False)

fid = np.sum((mu1 - mu2)**2) + np.trace(sigma1 + sigma2 - 2 * linalg.sqrtm(sigma1.dot(sigma2)).real)
print(f"✅ FID Score: {fid:.2f}")


✅ 5000 images generated from real latents
✅ Zipped outputs at: ldm1_outputs/generated_best.zip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 102.9 MB/s eta 0:00:00
Setting up [LPIPS] percept

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:03<00:00, 75.3MB/s]


Loading model from: /usr/local/lib/python3.11/dist-packages/lpips/weights/v0.1/alex.pth
✅ LPIPS Diversity Score: 0.6236


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:01<00:00, 69.9MB/s]


✅ FID Score: 8.43


In [ ]:
#GENERATING NEXT 5000 IMAGES
# ✅ GENERATION CODE MATCHING TRAINING QUALITY (REAL LATENTS + DDIM OPTION + ZIP + FID/LPIPS)
import torch, os, zipfile
from torchvision.utils import save_image
from tqdm import trange
from torch import nn
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import torchvision.transforms as T
import numpy as np
import torchvision.models as models
import torch.nn.functional as F
from scipy import linalg
from itertools import cycle

# ✅ CONFIG (same as training)
LATENT_DIM = 1024
TIMESTEPS = 1000
IMG_SIZE = 512
DATA_PATH = "/content/drive/MyDrive/INSECTDAMAGEWEBBER"

# ✅ DEVICE
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ Schedulers (same as training)
scheduler_betas = torch.linspace(1e-4, 0.02, TIMESTEPS).to(device)
scheduler_alphas = 1. - scheduler_betas
scheduler_alpha_hat = torch.cumprod(scheduler_alphas, dim=0)

# ✅ DATASET
class SingleClassImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.paths = [os.path.join(root_dir, f) for f in os.listdir(root_dir) if f.endswith((".png", ".jpg"))]
        self.transform = transform

    def __len__(self): return len(self.paths)

    def __getitem__(self, idx):
        img = Image.open(self.paths[idx]).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, 0

transform = T.Compose([
    T.Resize((IMG_SIZE, IMG_SIZE)),
    T.ToTensor(),
    T.Normalize([0.5]*3, [0.5]*3)
])

ref_transform = T.Compose([
    T.Resize((299, 299)),
    T.ToTensor(),
    T.Normalize([0.5]*3, [0.5]*3)
])

# Real data loader for FID
ref_dataset = SingleClassImageDataset(DATA_PATH, ref_transform)
ref_loader = DataLoader(ref_dataset, batch_size=32, shuffle=False)

# ✅ MODELS (same as training)
class Encoder(nn.Module):
    def __init__(self, latent_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1), nn.ReLU(),
            nn.Conv2d(64, 128, 4, 2, 1), nn.BatchNorm2d(128), nn.ReLU(),
            nn.Conv2d(128, 256, 4, 2, 1), nn.BatchNorm2d(256), nn.ReLU(),
            nn.Conv2d(256, 512, 4, 2, 1), nn.BatchNorm2d(512), nn.ReLU(),
            nn.Conv2d(512, 512, 4, 2, 1), nn.ReLU(),
            nn.Flatten(),
            nn.Linear(512 * 16 * 16, latent_dim)
        )
    def forward(self, x): return self.net(x)

class Decoder(nn.Module):
    def __init__(self, latent_dim):
        super().__init__()
        self.fc = nn.Linear(latent_dim, 512 * 16 * 16)
        self.net = nn.Sequential(
            nn.ConvTranspose2d(512, 512, 4, 2, 1), nn.BatchNorm2d(512), nn.ReLU(),
            nn.ConvTranspose2d(512, 256, 4, 2, 1), nn.BatchNorm2d(256), nn.ReLU(),
            nn.ConvTranspose2d(256, 128, 4, 2, 1), nn.BatchNorm2d(128), nn.ReLU(),
            nn.ConvTranspose2d(128, 64, 4, 2, 1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.ConvTranspose2d(64, 3, 4, 2, 1), nn.Tanh()
        )
    def forward(self, z): return self.net(self.fc(z).view(-1, 512, 16, 16))

class LatentUNet(nn.Module):
    def __init__(self, latent_dim, timesteps):
        super().__init__()
        self.time_embed = nn.Embedding(timesteps, latent_dim)
        self.fc1 = nn.Sequential(
            nn.Linear(latent_dim, 1024), nn.ReLU(),
            nn.Linear(1024, 1024), nn.ReLU(),
            nn.Linear(1024, latent_dim)
        )
    def forward(self, x, t):
        return self.fc1(x + self.time_embed(t))

# ✅ Instantiate models
encoder = Encoder(LATENT_DIM).to(device)
decoder = Decoder(LATENT_DIM).to(device)
unet = LatentUNet(LATENT_DIM, TIMESTEPS).to(device)

# ✅ Load trained weights
encoder.load_state_dict(torch.load("ldm_outputs/encoder_epoch1000.pth"))
decoder.load_state_dict(torch.load("ldm_outputs/decoder_epoch1000.pth"))
unet.load_state_dict(torch.load("ldm_outputs/unet_epoch1000.pth"))
encoder.eval(); decoder.eval(); unet.eval()

# ✅ OUTPUT FOLDER
output_dir = "ldm2_outputs/generated_best_2"
os.makedirs(output_dir, exist_ok=True)

# ✅ Generation function (real latents)
@torch.no_grad()
def generate_from_real_latents(encoder_model, decoder_model, dataloader, num_samples=5000, offset=5000):
    count = 0
    loop_loader = cycle(dataloader)  # Infinite loop over dataset
    for images, _ in loop_loader:
        z = encoder_model(images.to(device))
        imgs = decoder_model(z)
        imgs = (imgs + 1) / 2
        for j, img in enumerate(imgs):
            index = offset + count + j + 1
            save_image(img, os.path.join(output_dir, f"real_latent_{index:05}.png"))
        count += len(imgs)
        if count >= num_samples:
            break
    print(f"✅ {num_samples} images generated from real latents")

# ✅ Mode selector
mode = "real"
if mode == "real":
    generate_from_real_latents(encoder, decoder, DataLoader(SingleClassImageDataset(DATA_PATH, transform), batch_size=8, shuffle=True), num_samples=5000, offset=5000)

# ✅ ZIP GENERATED IMAGES
zip_path = output_dir + ".zip"
with zipfile.ZipFile(zip_path, 'w') as zipf:
    for root, _, files in os.walk(output_dir):
        for file in files:
            zipf.write(os.path.join(root, file), arcname=file)
print(f"✅ Zipped outputs at: {zip_path}")


✅ 5000 images generated from real latents
✅ Zipped outputs at: ldm2_outputs/generated_best_2.zip


In [ ]:
import shutil

# Path where you want to move the zip file in Drive
drive_dest = "/content/drive/MyDrive/INSECTDAM"

# Move zip from current dir to Drive
shutil.move(zip_path, drive_dest)

print("✅ Moved to Google Drive at:", drive_dest)


✅ Moved to Google Drive at: /content/drive/MyDrive/INSECTDAM
